<a href="https://colab.research.google.com/github/nielmelekh/IR-Final-Project/blob/main/IndexingTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: MapReduce & Web Graph

## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class :😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Staff will allow your notebook to **execute from start to finish for no more than 90 seconds**, then manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell.

**Submission**: Upload a zip file **with your ID as the file name** containing the colab notebook and the GCP notebook to the assignment on Moodle. The names of the files should be  in the format:  012345678_colab.ipynb and 012345678_gcp.ipynb.

# Tasks

In this assignment, we are going to use MapReduce in pySpark to scale up index creation. **The assignment consists of two parts, one in Google Colab and one in GCP**. In the first part (Colab), we will learn how to use MapReduce in PySpark, create an inverted index, and calculate PageRank. In the second part, we will use GCP to scale up index creation for the entire English Wikipedia.

You will learn how to work with PySpark, write MapReduce code that executes on a cluster of machines, and experience the power of parallel processing. **All tasks must be completed through the use of PySpark's MapReduce**.

To save you (and us) time, both in this assignment and in the final project, we are providing you with preprocessed files for the entire English wikipedia dump. Preprocessing was done, similar to Assignment #2, to extract article information from the XML, then remove the WikiMedia markup. The complete preprocessing code for one wiki dump can be seen [here](https://colab.research.google.com/drive/1xXc0YPU4BVd8CAjVNgICimW_d-Z7wryu?usp=sharing).


**By the end of this assignment, you will have:**

In Colab (65 Points)

1. (20 Points) Processed articles' body to produce posting lists (using PySpark and MapReduce).
2. (5 Points) Calculated document frequency (DF) in a  distributed fashion.
3. (20 Points) Partitioned the posting lists and wrote them to disk in parallel.
4. (20 Points) Constructed the links graph and calculated PageRank on it.
All of these will be based on a small subset of articles from one wiki dump file.

In GCP (35 Points):

1. (10 Points) Successfully started a Dataproc cluster, setup cloud storage bucket, and properly initialized cluster nodes.
2. (10 Points) Created an inverted index for all of English Wikipedia.
3. (10 Points) Successfully computed PageRank for the entire English Wikipedia.
4. (5 Points) Reporting: size of the data, size of index, time it took to build the index, time it took to compute PageRank, and the $ amount of credits it consumed.

Bonus (10 points) if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.







# Setup

## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

from inverted_index_colab import *

# define english stopwords from nltk Oct 2025
english_stopwords = frozenset([
    "during", "as", "whom", "no", "so", "shouldn't", "she's", "were", "needn", "then", "on",
    "should've", "once", "very", "any", "they've", "it's", "it", "be", "why", "ma", "over",
    "you'll", "they", "you've", "am", "before", "shan", "nor", "she'd", "because", "been",
    "doesn't", "than", "will", "they'd", "not", "those", "had", "this", "through", "again",
    "ours", "having", "himself", "into", "i'm", "did", "hadn", "haven", "should", "above",
    "we've", "does", "now", "m", "down", "he'd", "herself", "t", "their", "hasn't", "few",
    "and", "mightn't", "some", "do", "the", "we're", "myself", "i'd", "won", "after",
    "needn't", "wasn't", "them", "don", "further", "we'll", "hasn", "haven't", "out", "where",
    "mustn't", "won't", "at", "against", "shan't", "has", "all", "s", "being", "he'll", "he",
    "its", "that", "more", "by", "who", "i've", "o", "that'll", "there", "too", "they'll",
    "own", "aren't", "other", "an", "here", "between", "hadn't", "isn't", "below", "yourselves",
    "ve", "isn", "wouldn", "d", "we", "couldn", "ain", "his", "wouldn't", "was", "didn", "what",
    "when", "i", "i'll", "with", "her", "same", "you're", "yours", "couldn't", "for", "doing",
    "each", "aren", "which", "such", "mightn", "up", "mustn", "you", "only", "most", "of", "me",
    "she", "he's", "in", "a", "if", "but", "these", "him", "hers", "both", "my", "she'll", "re",
    "weren", "yourself", "is", "until", "weren't", "to", "are", "itself", "you'd", "themselves",
    "ourselves", "just", "wasn", "have", "don't", "ll", "how", "they're", "about", "shouldn",
    "can", "our", "we'd", "from", "it'd", "under", "while", "off", "y", "doesn", "theirs",
    "didn't", "or", "your", "it'll"
])


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP.

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met.

In [ ]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
# Java 17 for Spark 3.5.x


# test env- comment this !!!
# os.environ["JAVA_HOME"] = os.getenv("JAVA_HOME", f"{os.environ['CONDA_PREFIX']}")



# student env- uncomment this !!!

!apt -qq update -y
!apt -qq install -y openjdk-17-jdk-headless
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"  # NOT SURE IF TO COMMENT (MINE)
# (Optional) Ensure PySpark is 3.5.x
!pip -q install "pyspark>=3.5,<3.6"
# Install the Python wrapper for GraphFrames (PyPI has only 0.6; use GitHub)
!pip -q install "git+https://github.com/graphframes/graphframes.git#subdirectory=python"

1 package can be upgraded. Run 'apt list --upgradable' to see it.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
openjdk-17-jdk-headless is already the newest version (17.0.17+10-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, but you have pyspark 3.5.7 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro

In [ ]:
from pyspark.sql import SparkSession

GF_PKG = "graphframes:graphframes:0.8.3-spark3.5-s_2.12"

# Stop any old session quietly
try:
    spark.stop()
except:
    pass

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("IR-GraphFrames")
    .config("spark.jars.packages", GF_PKG)   # <-- brings in the JAR
    .config("spark.ui.port", "0")            # free UI port (avoid 'connection refused')
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)


sc = spark.sparkContext

from graphframes import GraphFrame
print("Spark:", spark.version, "| GraphFrames Python wrapper imported OK")

Spark: 3.5.7 | GraphFrames Python wrapper imported OK


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment.

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. Important: Ensure you have completed the **Google Storage** section of the PDF document before continuing with the next steps.
Notice that you have to change the project_id and data_bucket_name variables in the second cell below.

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [ ]:
# student env- uncomment this !!!
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 100

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except:
  pass

# Remove any existing path first
if "wikidata_preprocessed" in os.environ:
    del os.environ["wikidata_preprocessed"]

# test env- comment this !!!
# base_dir = '/Users/liorb/Library/CloudStorage/GoogleDrive-liorkob@post.bgu.ac.il/Shared drives/IR2025_2026_Staff/Assignments/coding_assignments'
# Set absolute path to the parquet file
# os.environ["LOCAL_PARQUET_PATH"] = os.path.join(base_dir, "wikidumps/multistream1_preprocessed.parquet")
# print(f"Using parquet file at: {os.environ['LOCAL_PARQUET_PATH']}")


Run this cell and follow the authorization link that appears. The link will take you to Google Cloud Platform (GCP) where you'll need to:

Sign in to your Google account
Copy the verification code provided
Return here to paste the code when prompted

In [ ]:
# student env- uncomment this !!!
import os
from pathlib import Path
from google.colab import auth
from google.cloud import storage

# Alternative authentication method
!gcloud auth login

# Set your project and bucket info
project_id = 'ir-assignment-3-480612'
data_bucket_name = '215329202_ir_assignment3_bucket'

# Configure project
!gcloud config set project {project_id}

# Create directory and copy file directly
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
    !mkdir -p wikidumps
    !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark.

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory.

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [ ]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [ ]:

# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd

## Word counts

### Term frequency

**YOUR TASK (10 POINTS):** Complete the implementation of `word_count`, which calculates the term-frequency (tf) for each word in a single document.  map function. The functions gets the text and the wiki id, calculates the tf and returns list of tuples in the form of (token, (wiki_id, tf)).

In [ ]:
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  text = text.lower()
  tokens = [token.group() for token in RE_WORD.finditer(text) if token.group() not in all_stopwords]
  # YOUR CODE HERE
  tf_count = Counter(tokens)
  return [(token, (id, tf_count[token])) for token in tf_count]

In [ ]:
# test for word count
text = "This is a test to check the words count function actually count words"
id = 0
word_count_results = word_count(text, id)
assert word_count_results[0][0] == 'test'
assert len(word_count_results) == 6
assert word_count_results[2][1][1] == word_count_results[2][1][1]
assert type(word_count_results[0][0]) == str
assert type(word_count_results[0][1]) == tuple


PySpark has a `map` trasformation that, just like we learned in class about Map in MapReduce, executes a given function on each element of an  RDD. It also has a `flatMap()` transformation that executes a given function on each element of an RDD, but afterwards flattens the results to create one big "list" instead multiple nested lists. For example, `flatMap` on the above `word_count` function will concatenate the tokens from multiple document into one big list rather than keeping separate list of tokens for each document like a regular `map` would produce.

In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

After `flatMap` is executed, the counts for a given token are likely to be scattered accross multiple machines/workers in the cluster. In order to create the posting lists for each token, we need a reduce operation that collects them into one list and sorts them by the document id (wiki_id).

Spark has several ways of reducing or aggergating values, but we will use `groupByKey`, which like Reduce in MapReduce, collects all the values for a given key into an iterable list of the values associated with that key. The result of `groupByKey` is an RDD with (key, value*s*) pairs. Then, we can call `map` to execute a function on each pair in that RDD or call `mapValues` to execute a function only on the values iterable.

**YOUR TASK (5 POINTS):** Complete the implementation of `reduce_word_counts`, which operates on the pairs returned by `word_count`. The functions gets a list of values (unsorted posting list) and sorts it.

In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key=itemgetter(0))

In [ ]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
print(postings.count())


152999


In [ ]:
## test reduce_word_count
token, posting_list = postings.take(1)[0]
assert type(token) == str
assert type(posting_list) == list
doc_id, tf = zip(*posting_list)
assert type(doc_id[0]) == int
assert type(tf[0]) == int
assert np.diff(doc_id).min() > 0
assert np.min(tf) > 0
assert postings.count() == 152999

### Document frequency and filteration

Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).

In [ ]:
postings_filtered = postings.filter(lambda x: len(x[1])>10)

Now, we would like to calculate the df for each word.

**YOUR TASK (5 POINTS):** Complete the implementation of `calculate_df` that takes a posting list RDD and returns an RDD where each element is a (token, df) pair.

In [ ]:
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    # YOUR CODE HERE
    return postings.map(lambda x: (x[0], len(x[1])))

In [ ]:
# global statistics
w2df = calculate_df(postings_filtered)
# Collect the first row and force its execution
first_row = w2df.take(1)
token, df = first_row[0]

# Continue with the rest of the operations
print(type(token), type(df))
assert type(token) == str
assert type(df) == int

# test min/max df values
w2df_dict = w2df.collectAsMap()
print(np.min(list(w2df_dict.values())), np.max(list(w2df_dict.values())))
assert np.min(list(w2df_dict.values())) == 11
assert np.max(list(w2df_dict.values())) == 819

# test select words
print(w2df_dict['first'], w2df_dict['many'], w2df_dict['used'])
assert w2df_dict['first'] == 805
assert w2df_dict['many'] == 670
assert w2df_dict['used'] == 648


<class 'str'> <class 'int'>
11 819
805 670 648


## Partitioning and writing the index

Now that we have the postings information we need to write it to disk. We would like to partition the tokens and posting lists into buckets so when we get a query, we know exactly in which bucket to look for its posting list. This can also help parallelize retrieval from the index if we have multiple machines answering queries.

Here, you are asked to do this partitioning/bucketing through another round of map and reduce operations, where the last step should be to write out each bucket into a separate inverted index on disk. Use the staff-provided function of  `write_a_posting_list` implemented in inverted_index_colab.py for the InvertedIndex class. Eventually, this partitioning and writing process needs to return information about where each posting list is located (file name and offset) for a token.

**YOUR TASK (20 POINTS):** Complete the implementation of `partition_postings_and_write` that partitions the posting list, writes out each bucket, and returns information about the location on disk of each posting list.

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  postings_buckets = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
  return postings_buckets.map(lambda x: InvertedIndex.write_a_posting_list(x))

The next cell uses the `collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary. We know in this case that the posting locations info is relatively small so this is not going to crash our master node.

In [ ]:
# test partitioning for the different buckets (10 points)
posting_locs_list = partition_postings_and_write(postings_filtered).collect()
assert len(posting_locs_list) == 124
assert len(list(Path('.').glob('*.bin'))) == 124
pl_lens = [len(pl) for pl in posting_locs_list]
assert np.max(pl_lens) == 141
assert np.min(pl_lens) == 91
assert int(np.median(pl_lens)) == 116

In [ ]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)
print(len(super_posting_locs))
assert super_posting_locs['try'][0][0][:3] == '105'
assert super_posting_locs['political'][0][0][:2] == '83'
assert 'hi' not in super_posting_locs
assert len(super_posting_locs) == 14519

14519


Putting everything together (posting locations, df) and test that the resulting index is correct.

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')

In [ ]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
## test the final index
pl = read_posting_list(inverted, 'clarity')
assert pl[-3] == (2204, 2)
assert len(pl) == 18

## PageRank

In this section, your task is to compute PageRank for wiki articles using the anchor text we extracted from the MediaWiki markdown. Let's select the 'id' and 'anchor_text' fields into a new RDD:

In [ ]:
pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd

**YOUR TASK (20 POINTS):** Complete the implementation of `generate_graph`, which generates the graph of internal wikipedia links (wiki articles pointing to other wiki articles). Use PySpark to compute the vertices (pages) and edges (links) of the graph. Multiple links from page A to page B need to be represented by a single edge (edges are not weighted). You may use the built-in function `distinct` of the RDD object to identify distinct elements, but be mindful of its impact on computation time.

In [ ]:
from pyspark.sql.types import StructType, StructField, LongType  # Don't forget
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE
  edges = pages.flatMap(lambda x: [(x[0], link[0]) for link in x[1]]).distinct()
  vertices = edges.flatMap(lambda x: [(x[0],), (x[1],)]).distinct()
  return edges, vertices

In [ ]:
# test graph construction efficiency (10 points)
t_start = time()
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(pages_links)
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()
graph_const_time = time() - t_start

In [ ]:
# test that the implementation is efficient (<20 seconds)
assert graph_const_time < 20


In [ ]:
# test that we get the right number of vertices (5 points)
assert v_cnt == 114966

In [ ]:
# test that we get the right number of edges (5 points)
assert e_cnt == 183727

Let's compute PageRank in a distributed fashion (using PySpark's GraphFrame package) and show the top 20 pages.

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
# pr.repartition(1).write.csv('pr', compression="gzip")
pr.repartition(1).write.mode('overwrite').csv('pr', compression="gzip")

pr.show()

+--------+------------------+
|      id|          pagerank|
+--------+------------------+
|  606848|1.9768910954777374|
|43432956|1.7183234055660317|
|37404646| 1.561180019985396|
|  148363| 1.531189053212204|
|     308| 1.523973241896932|
|     783|1.5205983727677175|
|   39235|1.5113438223085562|
|   30680| 1.506487061855029|
| 3434750|1.5042432651671351|
|   48518|1.5032093881505733|
| 5843419| 1.475564533072746|
|   25458|1.4651925575346503|
|     854| 1.433614708798739|
|   14532|1.4318560075868727|
|   11887| 1.422942493084949|
|   32927|1.4195131540605397|
| 8034453| 1.414325748041313|
|    1686|1.4005429901986137|
|   25507|1.3695924013144563|
|   61371|1.3682656561066355|
+--------+------------------+
only showing top 20 rows



**YOUR TASK (5 POINTS)**: Complete the implmentation of `get_top_pr_page_titles` to return the titles a few of the top pages as ranked by PageRank and printed above. To get the title for each page id listed above, navigate to the url https://en.wikipedia.org/?curid=wiki_id_here by replacing the `wiki_id_here` part with the page id. For example, if you navigate to https://en.wikipedia.org/?curid=25507 for the page with id 25507 you'd see that its title is 'Roman Empire'.

In [ ]:
def get_top_pr_page_titles():
    ''' Returns the title of the first, fourth, and fifth pages as ranked
        by PageRank, using the Wikipedia curid URL as specified.
    Returns:
    --------
      list of three strings.
    '''

    from pyspark.sql.functions import col
    import requests
    from bs4 import BeautifulSoup

    # YOUR CODE HERE
    headers = {
        'User-Agent': 'My assignment for IR course)'
    }
    selected_ranks = [1,4,5]
    url = "https://en.wikipedia.org/?curid="
    titles = []
    first_5 = pr.take(5)
    for rank in selected_ranks:
        response = requests.get(url + str(first_5[rank-1][0]), headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        titles.append(soup.title.string[:-12])
    return titles

In [ ]:
top_pr=get_top_pr_page_titles()
print(top_pr)
assert [_hash(x.lower()) for x in top_pr] == \
  ['52bb7dbc02', '3e920ea944', '3d007b19ee']

['Catholic Church', 'Ancient Greek', 'Aristotle']


Do the top ranked pages make sense to you? Why do you think these pages received such a high scores from PageRank?

The code to the GCP notebook is in the moodle under assignment 3. You will have to copy the code you completed here in the relevant sections.

In the current notebook we used MapReduce to create the index for only one wiki dump. Using GCP, we will be able to create the index on the entire english wikipedia. The GCP part of this assignment will be the first time we will work with the entire corpus. You will need to follow the notebook, complete the code as you did here (the same code you completed in this notebook can be used in the GCP notebook).
At the end of the GCP assignment you will have questions that you will need to answer - this are not the tests we used so far, so you should follow the instructions in the notebook.